In [16]:
import torch
import schnetpack as spk
import math
from schnetpack.datasets import QM9


import numpy as np
from numpy import savetxt

# Define Important Functions

In [17]:
def print_molecule(props):
    
    # load x,y,z coordinates tensors
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 
    print('')  
    return z

def hook_v0(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v0
    #Update the embedding_output variable to be equal to our output tensor
    v0=out_tensor 

def hook_v1(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v1
    #Update the embedding_output variable to be equal to our output tensor
    v1=out_tensor 

def hook_v2(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v2
    #Update the embedding_output variable to be equal to our output tensor
    v2=out_tensor  
    
def hook_emb(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global emb
    #Update the embedding_output variable to be equal to our output tensor
    emb=out_tensor 
#def convert_2D(number_of_atoms,rep):
#    layer = np.zeros((number_of_atoms,30))
#    for i in range(number_of_atoms):
#        for j in range(30):
#            layer[i][j] = rep[0][i][j]
#    return layer

def hook_representation(qm9data,checkpoint_path,split_file,number_of_inputs):


SyntaxError: unexpected EOF while parsing (<ipython-input-17-86eeb8b7f901>, line 59)

In [20]:
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)
checkpoint_path = '../../../../data/trained_models/qm9_i3_30f_10000_5000/trained.pth'
split_file='../../../../data/trained_models/qm9_i3_30f_10000_5000/split.npz'
number_of_inputs = 5000

# Load split file 
train, val, test = spk.data.train_test_split(qm9data,split_file=split_file)

# Load atom ref data 
atomrefs = qm9data.get_atomref(QM9.U0)
# Define SchNet representation model

schnet = spk.representation.SchNet(
    n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=3,
    cutoff=4. , cutoff_network=spk.nn.cutoff.CosineCutoff
)

# Define SchNet output model and property to be predicted

output_U0 = spk.atomistic.Atomwise(n_in=30, atomref=atomrefs[QM9.U0])

# Define atomistic model

model = spk.AtomisticModel(representation=schnet,output_modules=output_U0)

# Load saved checkpoint file
load_checkpoint = torch.load(checkpoint_path)

#qm9_i6_30f_20g-1000-500-4_300.pth
# load model's state dictionary from saved checkpoint
model.load_state_dict(load_checkpoint)

#set up device for forward pass
device='cpu'

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

datao = np.zeros((1,30))
datahae = np.zeros((1))
dataoae = np.zeros((1))
datah = np.zeros((1,30))
for idx in range(number_of_inputs):

    # load data for molecule
    at, props = qm9data.get_properties(idx)

    # print molecule for identification
    print(idx)
    z = print_molecule(props)
    number_of_atoms=len(z)

    # convert qm9 data to machine-readable form
    inputs = converter(at)

    #Instatiate layer output
    v0=None
    v1=None
    v2=None
    x=None       

    # Forward hook the model's interaction layer 
    model.representation.interactions[0].register_forward_hook(hook_v0)

    # Forward hook the model's interaction layer 
    model.representation.interactions[1].register_forward_hook(hook_v1)

    # Forward hook the model's interaction layer 
    model.representation.interactions[2].register_forward_hook(hook_v2)

    # Forward hook the model's interaction layer 
    model.representation.embedding.register_forward_hook(hook_emb)

    # Forward pass molecules through the model
    model(inputs)


    rep = emb + v0 + v1 + v2
    rows = np.zeros((number_of_atoms,30))
    for i in range(number_of_atoms):
        for j in range(30):
            rows[i][j] = rep[0][i][j]

    from schnetpack.atomistic.output_modules import yi

    yi=yi.detach().numpy()

    #save the vector of every oxygen atom encountered
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            datao = np.vstack((datao,rows[i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datah = np.vstack((datah,rows[i]))    
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datahae = np.vstack((datahae,yi[0][i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            dataoae = np.vstack((dataoae,yi[0][i]))            

+


None
0
C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613

1
N -0.03160822 0.021159047 0.060332563
H 0.026075294 -0.9904034 -0.029608395
H 0.9246072 0.35579658 -0.030988995
H -0.5114599 0.34058344 -0.77777386

2
O -0.056248687 0.035458494 0.0007260183
H 0.042878296 -0.9215089 -0.00534094
H 0.84990215 0.35871136 -0.0061824406

3
C 0.5995395 0.0 0.0
C -0.5995395 0.0 0.0
H -1.6616386 0.0 0.0
H 1.6616386 0.0 0.0

4
C -0.0075630867 0.5570853 0.003070365
N 0.008071566 -0.59453905 -0.0032767907
H -0.022041854 1.6235689 0.008948269

5
C -0.008137181 0.5992856 0.002790173
O 0.0081545 -0.60058945 -0.0028012523
H 0.9208008 1.2085854 -0.0010057696
H -0.9532692 1.1830921 0.012220519

6
C -0.010403622 0.7647314 0.004216852
C 0.010404125 -0.7647325 -0.004217135
H 1.0031731 1.1788927 -0.0032747546
H -0.53377575 1.1627601 -0.8713333
H -0.5169408 1.1533226 0.8938080

39
C 0.74676186 1.8234588 0.0026987367
C 0.71465826 0.29370835 -0.0053245095
C -0.7016382 -0.2650462 0.0052626343
O -0.61744595 -1.6824552 -0.003045157
H 1.7753277 2.1956031 -0.005183291
H 0.23768522 2.239137 -0.8741201
H 0.25471118 2.2295043 0.893636
H 1.2319598 -0.093814634 -0.8907419
H 1.2488378 -0.10337167 0.86572224
H -1.2367903 0.09749595 0.8990794
H -1.253763 0.106896155 -0.8742458
H -1.5111663 -2.0367522 0.0027662576

40
C -0.37670365 1.7365772 -0.58901405
C -0.35834232 0.21751666 -0.60221523
O 0.32608134 -0.23554656 0.54658806
C 0.3953768 -1.6371869 0.623925
H 0.6429466 2.1321328 -0.59214145
H -0.9014168 2.1197355 -1.4693707
H -0.8841458 2.1059532 0.30665258
H -1.3875031 -0.18209246 -0.6149588
H 0.13942331 -0.15579951 -1.5142956
H -0.6056665 -2.09662 0.6676761
H 0.92978466 -2.0706418 -0.23724067
H 0.9383948 -1.890225 1.5380305

41
O 0.88869995 1.1945848 0.27268523
C 0.8989689 -0.22466038 0.36188236
C -0.5202829 -0.7874435 0.31435534
O -1.2094069 -0.33747628 -0.8453615
H 0.7130

61
C -0.052667186 1.5881379 0.6377174
C -0.06880968 0.047822926 0.60453784
C -1.5006969 -0.5198824 0.64008904
C 0.6510138 -0.4480523 -0.5745721
N 1.2116657 -0.8334883 -1.5090418
H 0.96935123 1.9745116 0.63065225
H -0.5799721 1.9991958 -0.2278283
H -0.5515862 1.9412478 1.5447142
H 0.4745751 -0.32444516 1.4824102
H -2.0745227 -0.17650776 -0.2253631
H -1.4946266 -1.6124898 0.63469666
H -2.0082986 -0.1794794 1.5471232

62
C 0.5450326 1.5968091 -0.2746015
C 0.55363744 0.06308237 -0.24318773
N 1.3183624 -0.415496 0.9106862
C -0.8297439 -0.46547034 -0.28376332
N -1.9011331 -0.9008385 -0.2748727
H 1.5761107 1.9565881 -0.27789873
H 0.026943982 1.9674567 -1.1621372
H 0.037180185 1.9970266 0.60830957
H 1.0598638 -0.30253276 -1.1445831
H 0.85638237 -0.12812711 1.7697307
H 1.3371655 -1.4311808 0.9224594

63
C -0.024941536 1.5804 0.6270696
C -0.005323342 0.045704123 0.5899224
O -1.3089012 -0.5031127 0.7618272
C 0.63353485 -0.45244792 -0.6337418
C 1.121628 -0.8410348 -1.661492
H 0.9908436 1.9844332 0

82
C -0.18289934 1.6042753 -0.015727855
C -0.13390873 0.088573486 -0.0530308
C -1.3854471 -0.5669182 -0.6054168
C 1.1942666 -0.5692452 -0.3395213
C 0.6074971 -0.6225036 1.0531088
H 0.7425631 2.0246127 0.3902766
H -0.32560608 2.0201697 -1.02041
H -1.0120881 1.9578694 0.60887945
H -1.5794762 -0.24373159 -1.6352811
H -1.297035 -1.6578658 -0.60985535
H -2.2659814 -0.3059959 -0.00600338
H 1.2002164 -1.4826398 -0.92556787
H 2.0582016 0.066421434 -0.5048103
H 1.0757284 -0.0227479 1.8269489
H 0.2177659 -1.5718249 1.4062023

83
C -0.18294045 1.6065004 0.016762566
C -0.1734727 0.09359709 -0.0070987167
C -1.389369 -0.58661604 0.58319676
C 0.64613485 -0.6143415 -1.0023906
O 1.0822308 -0.50363994 0.35500774
H 0.7466906 2.0064352 -0.3950807
H -1.0227643 2.0004513 -0.56588155
H -0.28772292 1.9728564 1.0439292
H -2.2954245 -0.31317577 0.03158995
H -1.277363 -1.6731039 0.5553552
H -1.5291034 -0.28360415 1.6267126
H 0.36388972 -1.6169183 -1.3229653
H 1.227644 -0.046742838 -1.7285236

84
C -0.1565042 1.6

102
C -0.80937845 0.9742669 0.011615371
C 0.6849378 0.5722226 -4.0288487e-05
C 0.40674868 -0.48133644 1.0987701
C -0.68800884 -0.5684616 0.008507617
O 0.3708276 -0.45399666 -1.0226791
H -1.2084086 1.4612448 0.9054476
H -1.1783545 1.411413 -0.91678154
H 1.5690228 1.1897867 -0.16213736
H 1.1035367 -1.3198208 1.1230955
H 0.120888814 -0.12981416 2.0937688
H -1.4582707 -1.3253772 -0.14329211

103
C -0.80328614 -2.1615045 -0.003920335
C -0.76291984 -0.96196747 0.0021173009
C -0.73006284 0.501471 0.0096675195
C 0.6241135 1.0572659 -0.0002943403
C 1.7291875 1.5255594 -0.008332598
H -0.83366084 -3.2228887 -0.009300699
H -1.2858185 0.8821762 -0.8582107
H -1.2690846 0.87277085 0.89205325
H 2.7089858 1.9347441 -0.01545681

104
C -0.77726597 -2.179581 -0.004280723
C -0.75442594 -0.98053724 0.0019412723
C -0.7439033 0.47992676 0.009685928
C 0.61075735 1.0471761 -0.00020573997
N 1.6685902 1.5091121 -0.007841379
H -0.79277474 -3.2417822 -0.0098386975
H -1.2863955 0.8696644 -0.86104864
H -1.2696161 0.8

KeyboardInterrupt: 

In [19]:
print('DONE')            
savetxt('../../../data/dataO.csv',datao,delimiter=',')
savetxt('../../../data/dataH.csv',datah,delimiter=',')
savetxt('../../../data/hae.csv',datahae,delimiter=',')
savetxt('../../../data/oae.csv',dataoae,delimiter=',')


DONE


NameError: name 'datao' is not defined

DONE


ValueError: You have to supply either split sizes (num_train / num_val) or an npz file with splits.